
## How to prepare a dataset and submit a custom entity in Amazon Comprehend

This notebook covers how to prepare a training dataset for custom entities in Amazon Comprehend

More information on how to create a custom entity recognizer model can be found here.

https://docs.aws.amazon.com/comprehend/latest/dg/training-recognizers.html


In [28]:
# library imports
import re
import numpy as np
import pandas as pd
import matplotlib
import csv
import boto3

In this example we will be using the following tweet dataset. https://www.kaggle.com/thoughtvector/customer-support-on-twitter First lets get our data and process it to our needs

In [45]:
role = 'arn:aws:iam::951145066533:role/service-role/AmazonComprehendServiceRole-cmpdner'

In [29]:
CUSTOM_NER_PREFIX = 'TweetTelco'
BUCKET = 'comprehend-ner-20200602'
s3_file = 'data/tweet_telco.csv'
s3 = boto3.client('s3')
s3.download_file(BUCKET,s3_file,s3_file)


In [11]:
colnames=['text'] 
tweets = pd.read_csv('data/tweet_telco.csv',encoding='utf-8',names=colnames, header=None)
print(tweets.shape)
tweets.head()

(32716, 1)


,text
0,@sprintcare is the worst customer service | @1...
1,@sprintcare is the worst customer service | @1...
2,@sprintcare is the worst customer service | @1...
3,@115714 y’all lie about your “great” connectio...
4,"@115714 whenever I contact customer support, t..."


In order to create our dataset we need to provide an entity list for our new class named NEGATIVITY.

In order to find relevant entities, we used another blazingtext to find similar words using word2vec. See ./blazingtext_word2vec/blazingtext_word2vec_telco_tweets.ipynb for examples of retrieving such keywords.

In [12]:
negative_words = ['Really', 'cheated', 'annoyed', 'unhelpful', 'frustrated', 'upset' , 'unhappy', 'angry', 'badly', 'bad', 'surprised', 'sadly', 'dissatisfied', 'disappointed', 'disgusted']

df_entity_list = pd.DataFrame(negative_words, columns=['Text'])

Let's add another column with our class label. This is required part of the Amazon Comprehend training dataset.

More information can be found here.

https://docs.aws.amazon.com/comprehend/latest/dg/cer-entity-list.html

In [14]:
df_entity_list['Type'] = 'NEGATIVE'

Let's create a training file

In [19]:
tweets['text'].to_csv('data/raw_negative.csv', encoding='utf-8', index=False,header=False)

In [20]:
!head data/raw_negative.csv

"@sprintcare is the worst customer service | @115712 Can you please send us a private message, so that I can gain further details about your account?"
@sprintcare is the worst customer service | @115712 I would love the chance to review the account and provide assistance.
@sprintcare is the worst customer service | @115712 Hello! We never like our customers to feel like they are not valued.
"@115714 y’all lie about your “great” connection. 5 bars LTE, still won’t load something. Smh. | @115713 H there! We'd definitely like to work with you on this, how long have you been experiencing this issue? -AA"
"@115714 whenever I contact customer support, they tell me I have shortcode enabled on my account, but I have never in the 4 years I've tried https://t.co/0G98RtNxPK | @115715 Please send me a private message so that I can send you the link to access your account. -FR"
"@115913 @115911 just called in to switch from AT&amp;T. They wanted $75 to switch 3 phones! I said no way! Inconsistent m

Let's create the entity list file

In [30]:
df_entity_list.to_csv('data/entity_negative_list.csv', encoding='utf-8', index=False)

In [23]:
!head data/entity_negative_list.csv

Text,Type
Really,NEGATIVE
cheated,NEGATIVE
annoyed,NEGATIVE
unhelpful,NEGATIVE
frustrated,NEGATIVE
upset,NEGATIVE
unhappy,NEGATIVE
angry,NEGATIVE
badly,NEGATIVE


In [27]:
entity_file = 'data/entity_negative_list.csv'
s3.upload_file(entity_file, BUCKET, entity_file)

Let's create a test file from our original telco tweet dataset.

In [33]:
train_file = 'data/telco_negative_test.csv'
tweets['text'].tail(10000).to_csv(train_file, encoding='utf-8', index=False,header=False)
s3.upload_file(train_file, BUCKET, train_file)

In [61]:
test_file = 'data/telco_negative_test.csv'
tweets['text'].head(10000).to_csv(test_file, encoding='utf-8', index=False,header=False)
s3.upload_file(test_file, BUCKET, test_file)

### Training the custom NER Model

In [62]:


s3_entity_key = entity_file
s3_train_key = train_file
s3_test_key = test_file

prefix = CUSTOM_NER_PREFIX

#Create s3 paths
s3_train_data = 's3://{}/{}'.format(BUCKET, s3_train_key)
s3_train_entity = 's3://{}/{}'.format(BUCKET, s3_entity_key)
s3_test_data = 's3://{}/{}'.format(BUCKET, s3_test_key)
s3_output_test_data = 's3://{}/{}/test/{}'.format(BUCKET, prefix, "telco_test_output.json")
print('uploaded training data location: {}'.format(s3_train_data))




uploaded training data location: s3://comprehend-ner-20200602/data/telco_negative_test.csv


In [48]:
# Instantiate Boto3 Client
comprehend = boto3.client('comprehend', region_name='us-east-1')

custom_entity_request = {

      "Documents": { 
         "S3Uri": s3_train_data
      },
      "EntityList": { 
         "S3Uri": s3_train_entity
      },
      "EntityTypes": [ 
         { 
            "Type": "NEGATIVE"
         }
      ]
   
}


# Create a document classifier
id = str(datetime.datetime.now().strftime("%s"))
create_custom_entity_response = comprehend.create_entity_recognizer(
        RecognizerName = CUSTOM_NER_PREFIX+id, 
        DataAccessRoleArn = role,
        InputDataConfig = custom_entity_request,
        LanguageCode = "en"
)
print("Create response: %s\n", create_custom_entity_response)

Create response: %s
 {'EntityRecognizerArn': 'arn:aws:comprehend:us-east-1:951145066533:entity-recognizer/TweetTelco1591147636', 'ResponseMetadata': {'RequestId': '8d6f1b5e-4e35-4cab-b776-1fee58ee6ed6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8d6f1b5e-4e35-4cab-b776-1fee58ee6ed6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '106', 'date': 'Wed, 03 Jun 2020 01:27:16 GMT'}, 'RetryAttempts': 0}}


In [52]:
create_custom_entity_response['EntityRecognizerArn']

'arn:aws:comprehend:us-east-1:951145066533:entity-recognizer/TweetTelco1591147636'

In [59]:
# Check the status of the classifier
describe_response = comprehend.describe_entity_recognizer(
    EntityRecognizerArn=create_custom_entity_response['EntityRecognizerArn'])
print("Describe response: %s", describe_response['EntityRecognizerProperties']['Status'])

jobArn = create_custom_entity_response['EntityRecognizerArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_custom_recognizer = comprehend.describe_entity_recognizer(
        EntityRecognizerArn = jobArn
    )
    status = describe_custom_recognizer["EntityRecognizerProperties"]["Status"]
    print("Custom entity recognizer: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        break
        
    time.sleep(60)

Describe response: %s TRAINED
Custom entity recognizer: TRAINED


### Testing our Model

In [60]:
print(json.dumps(describe_custom_recognizer["EntityRecognizerProperties"]["RecognizerMetadata"]["EntityTypes"], indent=2, default=str))

[
  {
    "Type": "NEGATIVE",
    "EvaluationMetrics": {
      "Precision": 94.0,
      "Recall": 100.0,
      "F1Score": 96.90721649484536
    },
    "NumberOfTrainMentions": 353
  }
]


In [69]:
test_response = comprehend.start_entities_detection_job(
    InputDataConfig={
        'S3Uri': s3_test_data,
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': s3_output_test_data
    },
    DataAccessRoleArn=role,
    JobName='Custom_Negative_Test',
    EntityRecognizerArn=jobArn,
    LanguageCode='en'
)


Lets monitor the job

In [70]:
jobId = test_response['JobId']
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_job = comprehend.describe_entities_detection_job(
        JobId = jobId
    )
    status = describe_job["EntitiesDetectionJobProperties"]["JobStatus"]
    print("Job Status: {}".format(status))
    
    if status == "COMPLETED" or status == "FAILED":
        break
        
    time.sleep(60)

Job Status: SUBMITTED
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: IN_PROGRESS
Job Status: COMPLETED


In [75]:
job_key

'test/telco_test_output.json/951145066533-NER-be0965a2f7d85e75fc4c441f3160cd01/output/output.tar.gz'

In [73]:
#Download the test output to local machine
job_output = describe_job["EntitiesDetectionJobProperties"]["OutputDataConfig"]["S3Uri"]
path_prefix = 's3://{}/'.format(BUCKET)
job_key = os.path.relpath(job_output, path_prefix)

s3 = boto3.resource('s3')
s3.Bucket(BUCKET).download_file(job_key, 'output.tar.gz')

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
!tar xvzf output.tar.gz

In [ ]:
#Load all the Entities values in a list
import json

data = []
for line in open('output', 'r'):
    entities = json.loads(line)['Entities']
    if entities != None and len(entities) > 0:
        data.append(entities[0]['Text'])
    

# function to get unique values 
def unique(list1): 
      
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    for x in unique_list: 
        print(x), 
        
unique(data)

### Create and use an endpoint